In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class SkipGramModel(nn.Module):

    def __init__(self, emb_size_u, emb_size_v,emb_dimension):
        super(SkipGramModel, self).__init__()
        self.emb_dimension = emb_dimension
        self.u_embeddings = nn.Embedding(emb_size_u, emb_dimension, sparse=False)
        self.v_embeddings = nn.Embedding(emb_size_v, emb_dimension, sparse=False)
        self.init_emb()
        

    def init_emb(self):
        initrange = 0.5 / self.emb_dimension
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.uniform_(-0, 0)
        
    def forward(self, pos_u, pos_v,neg_v):
        emb_u = self.u_embeddings(pos_u)
        neg_v = neg_v.view(len(pos_u),-1)
        samples = torch.cat([pos_v,Variable(neg_v)],1)
        emb_v = self.v_embeddings(samples)
        score = torch.bmm(emb_v, emb_u.unsqueeze(2)).squeeze()
        score[:,1:]=score[:,1:].neg()
        score = F.logsigmoid(score)
        return -1 * (torch.sum(score))/ pos_u.size(0)

In [ ]:
from torch.utils.data import Dataset
from collections import defaultdict
from gensim.models.doc2vec import TaggedDocument
import random

class wDataSet(Dataset):
     def __init__(self, power=0.75, neg_samples=10):
        self.vocab_id = defaultdict(int)
        if vocab_id is not None:
            self.vocab_id = vocab_id
        self.vocab_ctx = defaultdict(int)
        self.index2id = dict()
        self.id2index = dict()
        self.index2ctx = dict()
        self.ctx2index = dict()
        self.build_vocab(tagged_docs)
        self.ctx_weights = torch.FloatTensor(self.make_cum_table(power))
        self.pairs = self.generate_pairs(tagged_docs)
        self.neg_samples=neg_samples


